carregando o dataset


In [3]:
!pip install kagglehub[pandas-datasets]


In [8]:
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

# Nome correto do arquivo
file_path = "Groceries_dataset.csv"

# Carregando o dataset
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "heeraldedhia/groceries-dataset",
    file_path
)

print("First 5 records:")
print(df.head())


<ipython-input-8-db9ca162c2dd>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:
   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk


Basket

In [10]:
!pip install mlxtend


In [9]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Verifique os dados
print(df.head())

# 2. Agrupar itens por transação (Member_number + Date → uma cesta)
df['Transaction'] = df['Member_number'].astype(str) + "_" + df['Date']
basket = df.groupby(['Transaction'])['itemDescription'].apply(list)

# 3. Transformar em formato de cesta binária (One-Hot Encoding)
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(basket).transform(basket)
df_encoded = pd.DataFrame(te_data, columns=te.columns_)

# 4. Aplicar Apriori para encontrar itemsets frequentes
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

# 5. Gerar regras de associação
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 6. Exibir as melhores regras ordenadas por lift
rules_sorted = rules.sort_values(by='lift', ascending=False)

# Mostrar as 10 melhores
print("\nTop 10 Regras de Associação:\n")
print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))


   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk

Top 10 Regras de Associação:

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [11]:
# Instale as dependências se estiver no Colab
# !pip install kagglehub[pandas-datasets] mlxtend

import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Carregar o dataset
file_path = "Groceries_dataset.csv"
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "heeraldedhia/groceries-dataset",
    file_path
)

print("Primeiros registros do dataset:")
print(df.head())

# Agrupar os itens por transação (cesta) usando Member_number + Date
df['Transaction'] = df['Member_number'].astype(str) + "_" + df['Date']
basket = df.groupby(['Transaction'])['itemDescription'].apply(list)

# Codificar transações em formato binário
te = TransactionEncoder()
te_data = te.fit(basket).transform(basket)
df_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Aplicar Apriori com suporte mínimo mais baixo
frequent_itemsets = apriori(df_encoded, min_support=0.002, use_colnames=True)

print("\nItemsets frequentes (top 10):")
print(frequent_itemsets.sort_values(by="support", ascending=False).head(10))

# Gerar regras de associação com lift >= 0.9
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.9)

# Filtrar regras com pelo menos 1 item em antecedente e consequente
rules = rules[(rules['antecedents'].apply(len) >= 1) &
              (rules['consequents'].apply(len) >= 1)]

# Mostrar as 10 melhores regras
print("\nTop 10 Regras de Associação (ordenadas por lift):")
print(rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))


<ipython-input-11-d93408bd7544>:12: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Primeiros registros do dataset:
   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk

Itemsets frequentes (top 10):
      support            itemsets
123  0.157923        (whole milk)
78   0.122101  (other vegetables)
93   0.110005        (rolls/buns)
104  0.097106              (soda)
124  0.085879            (yogurt)
94   0.069572   (root vegetables)
116  0.067767    (tropical fruit)
7    0.060683     (bottled water)
99   0.060349           (sausage)
24   0.053131      (citrus fruit)

Top 10 Regras de Associação (ordenadas por lift):
            antecedents          consequents   support  confidence      lift
33               (curd)            (sausage)  0.002941    0.087302  1.446615
32            (sausage)               (curd)  0.002941    0.048726  1.446615

In [2]:
import pandas as pd

# Instale a tabulate se ainda não tiver:

# Cria o DataFrame com os insights
df_insights = pd.DataFrame({
    "Insight": [
        "Whole milk é o item mais frequente",
        "curd → sausage (lift 1.45)",
        "brown bread → canned beer",
        "sausage é central em várias regras",
        "Vegetais + frankfurter"
    ],
    "Ação Recomendável": [
        "Destaque na loja/app",
        "Promoção cruzada",
        "Sugestão no carrinho",
        "Produto-chave para estratégias",
        "Perfil misto: saudável + rápido"
    ]
})

# Imprime o resumo visual em formato de tabela
print("\n🎯 Resumo Visual para Apresentação:")
print("-" * 40)
print(df_insights.to_markdown(index=False))



🎯 Resumo Visual para Apresentação:
----------------------------------------
| Insight                            | Ação Recomendável               |
|:-----------------------------------|:--------------------------------|
| Whole milk é o item mais frequente | Destaque na loja/app            |
| curd → sausage (lift 1.45)         | Promoção cruzada                |
| brown bread → canned beer          | Sugestão no carrinho            |
| sausage é central em várias regras | Produto-chave para estratégias  |
| Vegetais + frankfurter             | Perfil misto: saudável + rápido |
